In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [3]:
cdf = spark.read.csv('/dataframe/a_class_info.csv', header=True)
cdf.printSchema()
type(cdf)
cdf.show(3)

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: string (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)



pyspark.sql.dataframe.DataFrame

[Stage 1:>                                                          (0 + 1) / 1]

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     2B1| CCAAW|           18|Suburban| Non-public| Experimental|
+--------+------+-------------+--------+-----------+-------------+
only showing top 3 rows



In [4]:
cdf.columns

['class_cd', 'school', 'class_std_cnt', 'loc', 'school_type', 'teaching_type']

### 각 반 학생수(class_std_cnt)가 평균 반 학생수보다 많은 class의 data를 추출하시오

['class_cd', 'school', 'class_std_cnt', 'loc', 'school_type', 'teaching_type']
반코드        학교이름   반 학생수       지역    학교형태       교육형태

In [5]:
# view 생성
cdf.createOrReplaceTempView('class')

In [6]:
# where 절에서 평균 반 학생수를 구하고 그 값보다 많은 반을 추출
# 에러발생 : where절에서는 집계함수 사용 불가
# spark.sql('''
#     select * from class
#     where class_std_cnt >= avg(class_std_cnt) and class_std_cnt is not null
# ''')

- subquery 사용
  1. subquery 이용 평균 반 학생수를 전달하기
  2. where 절에서 반 학생수가 1에서 전달받은 값보다 큰 data 추출

In [7]:
# 1. subquery - 평균 반 학생수 select 하는 쿼리(집계함수는 select 절에만 사용가능)
spark.sql('select avg(class_std_cnt) from class where class_std_cnt is not null').show()

+------------------+
|avg(class_std_cnt)|
+------------------+
|21.828282828282827|
+------------------+



In [8]:
#2. 1의 query를 where에 비교 대상으로 사용(subquery 적용)
spark.sql('''
    select * from class
    where (class_std_cnt >= (select avg(class_std_cnt) from class)) and (class_std_cnt is not null)
''')

DataFrame[class_cd: string, school: string, class_std_cnt: string, loc: string, school_type: string, teaching_type: string]

### DF method  사용

In [9]:
# cdf.select('*')\
#    .where(cdf.class_std_cnt >= avg(cdf.class_std_cnt))
# where절에서 집계함수 사용해서 에러 발생, avg(cdf.class_std_cnt)이 코드가 값으로 명시가 되어야 함

In [10]:
type(cdf.select(avg('class_std_cnt'))) # df
cdf.select(avg('class_std_cnt')).collect()
cdf.select(avg('class_std_cnt')).collect()[0]
cdf.select(avg('class_std_cnt')).collect()[0][0]
# cdf.select(avg('class_std_cnt')).collect()[0][0]

pyspark.sql.dataframe.DataFrame

[Row(avg(class_std_cnt)=21.828282828282827)]

Row(avg(class_std_cnt)=21.828282828282827)

21.828282828282827

In [11]:
# df method 사용
cdf.select('*')\
   .where(cdf.class_std_cnt >= cdf.select(avg('class_std_cnt')).collect()[0][0]).show(2)

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     1Q1| CUQAM|           28|Urban|     Public|     Standard|
|     BFY| CUQAM|           27|Urban|     Public|     Standard|
+--------+------+-------------+-----+-----------+-------------+
only showing top 2 rows



### 복잡한 연산도 해보자

#### case 0. 학교가 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이를 구해보자

In [12]:
cdf.show(1)

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     6OL| ANKYI|           20|Urban| Non-public|     Standard|
+--------+------+-------------+-----+-----------+-------------+
only showing top 1 row



#### count("컬럼")
- 중복 포함 개수
    - 
#### count_distinct("컬럼")
- 중복을 제외하고 개수
    <!-- - ex. 각 지역별 학교의 수 -->

In [13]:
# 학교가 가장 많이 위치한 지역의 학교수와 가장 적게 위치한 지역의 학교수 구해 리스트로 반환
cdf.where(cdf.loc.isNotNull()) \
    .groupby(cdf.loc) \
    .agg(count("school").alias("school_cnt")) \
    .agg(max("school_cnt").alias("maxCnt"), min("school_cnt").alias("minCnt")) \
    .show()

+------+------+
|maxCnt|minCnt|
+------+------+
|    37|    26|
+------+------+



In [14]:
school_cnt = cdf.where(cdf.loc.isNotNull()) \
    .groupby(cdf.loc) \
    .agg(count_distinct("school").alias("school_cnt")) \
    .agg(max("school_cnt").alias("maxCnt"), min("school_cnt").alias("minCnt"))
school_cnt.collect()
school_cnt.collect()[0]
school_cnt.collect()[0][0]
school_cnt.collect()[0][1]

[Row(maxCnt=9, minCnt=7)]

Row(maxCnt=9, minCnt=7)

9

7

In [15]:
[cnt for cnt in
    cdf.where(cdf.loc.isNotNull()) \
    .groupby(cdf.loc) \
    .agg(count_distinct("school").alias("school_cnt")) \
    .agg(max("school_cnt").alias("maxCnt"), min("school_cnt").alias("minCnt")) \
    .collect()[0]
]

[9, 7]

In [16]:
cdf.where(cdf.loc.isNotNull()) \
    .groupby(cdf.loc) \
    .agg(sum("class_std_cnt").alias("loc_std_cnt")) \
    .where(count_distinct("school").isin([9, 7])) \
    .select("*") \
    .show()

+--------+-----------+
|     loc|loc_std_cnt|
+--------+-----------+
|   Urban|      906.0|
|Suburban|      717.0|
|   Rural|      538.0|
+--------+-----------+



- column 객체.isin([])
    - column객체의 값을 함수 내부의 집합데이터가 포함하ㅏ고 있는지 확인해서 T/F 반환하는 함수

In [17]:
# 학교가 가장 많이 위치한 지역의 총학생수와 학교가 가장 적게 위치한 지역의 총 학생수 구하기
#  == 지역의 학교수가 9개 이거나 지역의 학교수가 7개인 지역의 지역별 총 학생수

# 가장 많거나 적은 지역이 2개 이상이면 총 학생수가 적은 지역을 학교수가 가장 적은 지역으로 결정, 많은 지역은 학생수가 많은 지역을 가장 많은 지역으로 결정

cdf.where(cdf.loc.isNotNull()) \
    .groupby(cdf.loc) \
    .agg(sum("class_std_cnt").alias("loc_std_cnt")) \
    .where(count_distinct("school").isin(
                [cnt for cnt in 
                    cdf.where(cdf.loc.isNotNull()) \
                    .groupby(cdf.loc) \
                    .agg(count_distinct("school").alias("school_cnt")) \
                    .agg(max("school_cnt").alias("maxCnt"), min("school_cnt").alias("minCnt")) \
                    .collect()[0]
                ]
            )
          ) \
    .select(max("loc_std_cnt") - min("loc_std_cnt")).alias("학생 수 차이") \
    .show()

+-------------------------------------+
|(max(loc_std_cnt) - min(loc_std_cnt))|
+-------------------------------------+
|                                368.0|
+-------------------------------------+



In [18]:
print("학교가 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이 구하기")

학교가 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이 구하기


In [19]:
# from절에 subquery 사용
# 지역별 학교수 최대/최솟값

- spark.sql

In [20]:
# 각 지역별 학교수
spark.sql('''
    select count(distinct school) as cntSCH
    from class
    where loc is not null
    group by loc
''').show()

+------+
|cntSCH|
+------+
|     9|
|     7|
|     7|
+------+



In [21]:
# 지역별 학교수 최대/최소값
spark.sql('''
select max(cntSCH) as max, min(cntSCH) as min
from (
    select count(distinct school) as cntSCH
    from class
    where loc is not null
    group by loc
    )
''').show()

+---+---+
|max|min|
+---+---+
|  9|  7|
+---+---+



#### df 다중열
- in 연산자를 사용하려면 다중열 비교 주체가 두개여야 비교 가능
- 다중행 in : ex. where school in ["abc", "def", "ghi"]
- 다중열 in : ex. where school, class in [["abc"], ["def"]]
    - school컬럼값이 첫번째 열 "abc"에 있는지, class컬럼값이 두번째 열 "def"에 있는지 확인

In [22]:
print("학교가 가장 많이 위치한 지역의 총학생수와 학교가 가장 적게 위치한 지역의 총 학생수 구하기")
# having절에 서브쿼리
# having절은 groupby에 종속되어서 집계함수 사용가능
spark.sql('''
    select sum(class_std_cnt)
    from class
    where loc is not null
    group by loc
    having count(distinct school) = (select max(cntSCH) as max
                                    from (
                                        select count(distinct school) as cntSCH
                                        from class
                                        where loc is not null
                                        group by loc
                                        )
                                    ) or
            count(distinct school) = (select min(cntSCH) as min
                                    from (
                                        select count(distinct school) as cntSCH
                                        from class
                                        where loc is not null
                                        group by loc
                                        )
                                    )
''').show()

학교가 가장 많이 위치한 지역의 총학생수와 학교가 가장 적게 위치한 지역의 총 학생수 구하기
+------------------+
|sum(class_std_cnt)|
+------------------+
|             906.0|
|             717.0|
|             538.0|
+------------------+



In [23]:
print("학교가 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이 구하기")


학교가 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이 구하기


#### case 1. 소속된 반의 개수가 3개 이상인 학교들 중 학생 숫자가 가장 적은 학교의 학생 수를 구해보자
#### 단, 학교가 null인 데이터는 제외한다

In [24]:
# from절 subquery : inline view
# 학급수가 3개 이상인 학교와 학급 수, 총 학생수
spark.sql('''
    select school, count(class_cd) as `학급수`, sum(class_std_cnt) as `학생수`
    from class
    where school is not null
    group by school
    having count(class_cd) >= 3
''').show(1)

+------+------+------+
|school|학급수|학생수|
+------+------+------+
| VHDHF|     3|  51.0|
+------+------+------+
only showing top 1 row



In [25]:
spark.sql('''
    select min(`학생수`) as `최소학생수`
    from(
        select school, count(class_cd) as `학급수`, sum(class_std_cnt) as `학생수`
        from class
        where school is not null
        group by school
        having count(class_cd) >= 3)
''').show(1)

+----------+
|최소학생수|
+----------+
|      46.0|
+----------+



### DF method

In [26]:
# 학급수가 3개 이상인 학교들의 총 학생수
cdf.groupby(cdf.school) \
    .agg(sum("class_std_cnt").alias("std_cnt")) \
    .where((count(cdf.class_cd) >= 3) & (cdf.school.isNotNull())) \
    .show()

+------+-------+
|school|std_cnt|
+------+-------+
| VHDHF|   51.0|
| LAYPA|   57.0|
| GOOBU|  158.0|
| UUUQX|   84.0|
| CIMBB|   74.0|
| UKPGS|  128.0|
| UAGPU|   87.0|
| CCAAW|  109.0|
| FBUMG|   46.0|
| ZOWMK|  117.0|
| ZMNYA|   69.0|
| QOQTS|  149.0|
| CUQAM|  107.0|
| OJOBU|   81.0|
| GOKXL|   64.0|
| GJJHK|  118.0|
| KZKKE|  111.0|
| DNQDD|  122.0|
| VKWQH|  100.0|
| IDGFP|   94.0|
+------+-------+
only showing top 20 rows



In [27]:
# 학급수가 3개 이상인 학교들의 총 학생수 중 최솟값
cdf.groupby(cdf.school) \
    .agg(sum("class_std_cnt").alias("std_cnt")) \
    .where((count(cdf.class_cd) >= 3) & (cdf.school.isNotNull())) \
    .select(min("std_cnt").alias("최소학생수")) \
    .show()

+----------+
|최소학생수|
+----------+
|      46.0|
+----------+



#### case 1_1. 학급수가 3개 이상인 학교들 중 학생 숫자가 가장 적은 학교의 학생 수와 학교명을 추출하자
#### 단, 학교가 null인 데이터는 제외한다

### sql query

In [28]:
cdf.createOrReplaceTempView("classV")

In [29]:
# 학급수가 3개 이상인 학교와 학급 수
spark.sql('''
    select school, count(class_cd)
    from classV
    where school is not null
    group by school
    having count(class_cd) >= 3
''').show(1)

+------+---------------+
|school|count(class_cd)|
+------+---------------+
| VHDHF|              3|
+------+---------------+
only showing top 1 row



In [30]:
# 학급수가 3개 이상인 학교와 학급 수, 학생수
spark.sql('''
    select school, count(class_cd) as `학급수`, sum(class_std_cnt) as `학생수`
    from classV
    where school is not null
    group by school
    having count(class_cd) >= 3
''').show(1)

+------+------+------+
|school|학급수|학생수|
+------+------+------+
| VHDHF|     3|  51.0|
+------+------+------+
only showing top 1 row



In [31]:
# 학급수가 3개 이상인 학교 중 학생수의 최솟값
spark.sql('''
    select min(`학생수`)
    from(
        select school, count(class_cd) as `학급수`, sum(class_std_cnt) as `학생수`
        from classV
        where school is not null
        group by school
        having count(class_cd) >= 3)
''').show(1)

+-----------+
|min(학생수)|
+-----------+
|       46.0|
+-----------+



In [32]:
# 학급수가 3개 이상인 학교 중 학생수가 최솟값인 학교와 그 학생 수
spark.sql('''
    select school as `최소학생 수인 학교`, sum(class_std_cnt)  as `학생수`
    from classV
    group by school
    having sum(class_std_cnt) == (select min(`학생수`)
                                from(
                                    select school, count(class_cd) as `학급수`, sum(class_std_cnt) as `학생수`
                                    from classV
                                    where school is not null
                                    group by school
                                    having count(class_cd) >= 3))
''').show(1)

+------------------+------+
|최소학생 수인 학교|학생수|
+------------------+------+
|             FBUMG|  46.0|
+------------------+------+



### df moethod 사용

In [33]:
cdf.groupby(cdf.school) \
    .agg(sum("class_std_cnt").alias("std_cnt")) \
    .where((count(cdf.class_cd) >= 3) & (cdf.school.isNotNull())) \
    .select(min("std_cnt").alias("최소학생수")) \
    .show()

+----------+
|최소학생수|
+----------+
|      46.0|
+----------+



In [34]:
cdf.groupby(cdf.school) \
    .agg(sum("class_std_cnt").alias("std_cnt")) \
    .where((count(cdf.class_cd) >= 3) & (cdf.school.isNotNull())) \
    .select(min("std_cnt").alias("최소학생수")) \
    .collect()[0][0]

46.0

In [35]:
cdf.groupby(cdf.school) \
    .agg(sum("class_std_cnt").alias("std_cnt")) \
    .where(cdf.school.isNotNull()) \
    .where(col("std_cnt") == (
                         cdf.groupby(cdf.school) \
                            .agg(sum("class_std_cnt").alias("std_cnt")) \
                            .where((count(cdf.class_cd) >= 3) & (cdf.school.isNotNull())) \
                            .select(min("std_cnt").alias("최소학생수")) \
                            .collect()[0][0]
                             )
    ).show()

+------+-------+
|school|std_cnt|
+------+-------+
| FBUMG|   46.0|
+------+-------+



### case2. 1.지역에 따른 학교로 분류하고 분류된 학교의 class_cd가 2개 초과인 학교별로 각반의 학생수가 가장 작은 반의 학생수를 구하시오
### 위에서 구한 학생수중 가장 큰 값은 얼마인가?
1. 지역에 따른 학교로 분류하고 학교의 class_cd가 2개 초과인 학교들을 추출
2. 추출된 학교들에서 학생수가 가장 작은 반의 학생수 추출
3. 2번에서 추출된 학생수들 중 가장 큰 수 추출

### sql query 활용

In [37]:
# 참고
# Urban 지역의 학교 VVTVA는 반수가 4개고 각 반 학생수 중 가장 작은 수는 25
cdf.select('*').where((cdf.school=='VVTVA') & (cdf.loc=='Urban')).show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     7BL| VVTVA|           29|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     TB5| VVTVA|           25|Urban|     Public|     Standard|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



### sql query

In [39]:
# 1. 지역에 따른 학교로 분류하고 학교의 class_cd가 2개 초과인 학교들을 추출 - 지역에 따른 학교별 반 개수
spark.sql('''
    select loc, school, count(class_cd) as cnt
    from classV
    group by loc, school
    having count(class_cd) > 2
''').show()

+--------+------+---+
|     loc|school|cnt|
+--------+------+---+
|   Rural| VHDHF|  3|
|   Urban| GOOBU|  6|
|Suburban| ZMNYA|  3|
|   Urban| ZOWMK|  4|
|    NULL|  NULL|  3|
|Suburban| UUUQX|  5|
|Suburban| DNQDD|  5|
|   Urban| CUQAM|  4|
|   Urban| IDGFP|  5|
|   Rural| GOKXL|  3|
|   Rural| KZKKE|  5|
|   Rural| VKWQH|  5|
|Suburban| CCAAW|  6|
|   Rural| LAYPA|  3|
|   Urban| QOQTS|  6|
|Suburban| UAGPU|  4|
|   Rural| FBUMG|  3|
|   Urban| VVTVA|  4|
|Suburban| UKPGS|  6|
|Suburban| GJJHK|  5|
+--------+------+---+
only showing top 20 rows



In [128]:
# Urban 지역의 학교 VVTVA는 반수가 4개 반 학생수 중 가장 작은 수는 25
cdf.select('*').where((cdf.school=='VVTVA') & (cdf.loc=='Urban')).show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     7BL| VVTVA|           29|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     TB5| VVTVA|           25|Urban|     Public|     Standard|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



In [40]:
# 2. 추출된 학교들에서 학생수가 가장 작은 반의 학생수 추출 단, school값이  null 인 데이터 제외
spark.sql('''
    select loc, school, count(class_cd) as cnt, min(class_std_cnt) as minStd
    from classV
    group by loc, school
    having count(class_cd) > 2 and school is not null
''').show()

+--------+------+---+------+
|     loc|school|cnt|minStd|
+--------+------+---+------+
|   Rural| FBUMG|  3|    14|
|   Rural| GOKXL|  3|    19|
|   Rural| KZKKE|  5|    20|
|   Rural| LAYPA|  3|    17|
|   Rural| OJOBU|  4|    17|
|   Rural| VHDHF|  3|    15|
|   Rural| VKWQH|  5|    18|
|Suburban| CCAAW|  6|    15|
|Suburban| DNQDD|  5|    20|
|Suburban| GJJHK|  5|    21|
|Suburban| UAGPU|  4|    21|
|Suburban| UKPGS|  6|    18|
|Suburban| UUUQX|  5|    15|
|Suburban| ZMNYA|  3|    22|
|   Urban| CIMBB|  4|    17|
|   Urban| CUQAM|  4|    24|
|   Urban| GOOBU|  6|    24|
|   Urban| IDGFP|  5|    17|
|   Urban| QOQTS|  6|    22|
|   Urban| VVTVA|  4|    25|
+--------+------+---+------+
only showing top 20 rows



In [42]:
# 3. 2번에서 추출된 학생수들 중 가장 큰 수 추출
spark.sql('''
    select max(minStd) as maxMinStd
    from (
        select loc, school, count(class_cd) as cnt, min(class_std_cnt) as minStd
        from classV
        group by loc, school
        having count(class_cd) > 2 and school is not null
    )
''').show()
# 즉, Urban지역의 ZOWMK 학교는 한반의 학생수 27명이 가장 작은데, 다른 지역의 학교들의 학생수가 가장 작은 반은 27명보다 작다

+---------+
|maxMinStd|
+---------+
|       27|
+---------+



### df의 메서드 활용

In [48]:
# Urban 지역의 학교 VVTVA는 반수가 4개 반 학생수 중 가장 작은 수는 25
cdf.select("*") \
    .where((cdf.school=="VVTVA") & (cdf.loc == "Urban")) \
    .show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     7BL| VVTVA|           29|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     TB5| VVTVA|           25|Urban|     Public|     Standard|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



In [49]:
# 1. 지역에 따른 학교로 분류하고 학교의 class_cd가 2개 초과인 학교들을 추출
cdf.groupby("loc", "school") \
    .agg(count("class_cd").alias("cnt_cd")) \
    .where((col("cnt_cd") > 2) & (col("school").isNotNull())) \
    .show()

+--------+------+------+
|     loc|school|cnt_cd|
+--------+------+------+
|   Rural| VHDHF|     3|
|   Urban| GOOBU|     6|
|Suburban| ZMNYA|     3|
|   Urban| ZOWMK|     4|
|Suburban| UUUQX|     5|
|Suburban| DNQDD|     5|
|   Urban| CUQAM|     4|
|   Urban| IDGFP|     5|
|   Rural| GOKXL|     3|
|   Rural| KZKKE|     5|
|   Rural| VKWQH|     5|
|Suburban| CCAAW|     6|
|   Rural| LAYPA|     3|
|   Urban| QOQTS|     6|
|Suburban| UAGPU|     4|
|   Rural| FBUMG|     3|
|   Urban| VVTVA|     4|
|Suburban| UKPGS|     6|
|Suburban| GJJHK|     5|
|   Rural| OJOBU|     4|
+--------+------+------+
only showing top 20 rows



In [50]:
# Urban 지역의 학교 VVTVA는 반수가 4개 반 학생수 중 가장 작은 수는 25
cdf.select("*") \
    .where((cdf.school=="VVTVA") & (cdf.loc == "Urban")) \
    .show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     7BL| VVTVA|           29|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     TB5| VVTVA|           25|Urban|     Public|     Standard|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



In [51]:
# 2. 추출된 학교들에서 학생수가 가장 작은 반의 학생수 추출
# 지역에따른 학교의 반들에 대해 가장 학생수가 작은반의 학생수
cdf.groupby("loc", "school") \
    .agg(count("class_cd").alias("cnt_cd"), min("class_std_cnt").alias("minStd")) \
    .where((col("cnt_cd") > 2) & (col("school").isNotNull())) \
    .show()

+--------+------+------+------+
|     loc|school|cnt_cd|minStd|
+--------+------+------+------+
|   Rural| FBUMG|     3|    14|
|   Rural| GOKXL|     3|    19|
|   Rural| KZKKE|     5|    20|
|   Rural| LAYPA|     3|    17|
|   Rural| OJOBU|     4|    17|
|   Rural| VHDHF|     3|    15|
|   Rural| VKWQH|     5|    18|
|Suburban| CCAAW|     6|    15|
|Suburban| DNQDD|     5|    20|
|Suburban| GJJHK|     5|    21|
|Suburban| UAGPU|     4|    21|
|Suburban| UKPGS|     6|    18|
|Suburban| UUUQX|     5|    15|
|Suburban| ZMNYA|     3|    22|
|   Urban| CIMBB|     4|    17|
|   Urban| CUQAM|     4|    24|
|   Urban| GOOBU|     6|    24|
|   Urban| IDGFP|     5|    17|
|   Urban| QOQTS|     6|    22|
|   Urban| VVTVA|     4|    25|
+--------+------+------+------+
only showing top 20 rows



In [52]:
# 3. 2번에서 추출된 학생수들 중 가장 많은 수 추출
# # 지역에따른 학교의 반들에 대해 가장 학생수가 작은반의 학생수에서 가장 큰 수 
# 즉, Urban지역의 ZOWMK 학교는 한반의 학생수 27명이 가장 작은데, 다른 지역의 학교들의 학생수가 가장 작은 반은 27명보다 작다
cdf.groupby("loc", "school") \
    .agg(count("class_cd").alias("cnt_cd"), min("class_std_cnt").alias("minStd")) \
    .where((col("cnt_cd") > 2) & (col("school").isNotNull())) \
    .select(max(col("minStd")).alias("maxMinStd")) \
    .show()

+---------+
|maxMinStd|
+---------+
|       27|
+---------+



### case3. 지역에 따른 학교로 분류하고 분류된 학교의 class_cd가 2개 초과인 학교에서 학교별 가장 작은 학생수들을 추출 그 중에서 가장 큰 수를 구하시오(27)
- case 2번에서 진행
- 구한 수보다 학생수가 더 많은 반의 코드과 학생수를 cdf 전체 데이터에서 추출하시오

#### sql 쿼리

In [56]:
spark.sql('select count(*) from class').show()

spark.sql('''
    select class_cd as class, class_std_cnt as student
    from classV
    where class_std_cnt > (
                            select max(minStd) as maxMinStd
                                from (
                                select loc, school, count(class_cd) as cnt, min(class_std_cnt) as minStd
                                from classV
                                group by loc, school
                                having count(class_cd) > 2 and school is not null
                                )
                            )
''').show()
# Urban지역의 ZOWMK 학교의 학생수가 가장 작은반보다 학생수가 많은 반은  전체 데이터 102개의 class 중 13 클래스이다

+--------+
|count(1)|
+--------+
|     102|
+--------+

+-----+-------+
|class|student|
+-----+-------+
|  1Q1|     28|
|  OMI|     28|
|  ROP|     28|
|  18K|     31|
|  HKF|     28|
|  0N7|     28|
|  SUR|     28|
|  7BL|     29|
|  A93|     30|
|  YTB|     30|
|  Q0E|     30|
|  QA2|     30|
|  ZBH|     30|
+-----+-------+



#### df 메서드

In [69]:
cdf.select("class_cd", "class_std_cnt") \
    .where((cdf.class_std_cnt) > (
                                cdf.groupby("loc", "school") \
                                .agg(count("class_cd").alias("cnt_cd"), min("class_std_cnt").alias("minStd")) \
                                .where((col("cnt_cd") > 2) & (col("school").isNotNull())) \
                                .select(max(col("minStd")).alias("maxMinStd")) \
                                .collect()[0][0])).show()
# Urban지역의 ZOWMK 학교의 학생수가 가장 작은반보다 학생수가 많은 반은 전체 데이터 102개의 class 중 13 클래스이다

+--------+-------------+
|class_cd|class_std_cnt|
+--------+-------------+
|     1Q1|           28|
|     OMI|           28|
|     ROP|           28|
|     18K|           31|
|     HKF|           28|
|     0N7|           28|
|     SUR|           28|
|     7BL|           29|
|     A93|           30|
|     YTB|           30|
|     Q0E|           30|
|     QA2|           30|
|     ZBH|           30|
+--------+-------------+



#### case 4. 시골지역의 사립학교중 표준교육을 진행하는 학급들의 평균 학생수보다 학생수가 더 많은 도시 지역의 공립학교면서 특수교육을 진행하는 학교의 모든 정보를 추출하시오

In [70]:
cdf.columns
# loc : Urban, subUrban, Rural
# school_type : Public, Non-public
# teachin_type : Standard, Experimental

['class_cd', 'school', 'class_std_cnt', 'loc', 'school_type', 'teaching_type']

### sql query

In [75]:
# 시골지역의 사립학교중 표준교육을 진행하는 학교들의 평균 학생수
spark.sql('''
    select avg(class_std_cnt)
    from classV
    where loc=="Rural" 
    and school_type=="Non-public" 
    and teaching_type=="Standard"
''').show()

+------------------+
|avg(class_std_cnt)|
+------------------+
|              17.0|
+------------------+



In [76]:
# 시골지역의 사립학교중 표준교육을 진행하는 학교의 학급들의 평균 학생수보다 학생수가 더 많은 도시 지역의 공립학교면서 특수교육을 진행하는 학교의 모든 정보를 추출하시오
spark.sql('''
    select *
    from classV
    where loc=="Urban" 
    and school_type=="Public" 
    and teaching_type=="Experimental"
    and class_std_cnt > (
                        select avg(class_std_cnt)
                        from classV
                        where loc=="Rural" 
                        and school_type=="Non-public" 
                        and teaching_type=="Standard"
                        )
''').show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     X6Z| CUQAM|           24|Urban|     Public| Experimental|
|     W8A| GOOBU|           26|Urban|     Public| Experimental|
|     0N7| QOQTS|           28|Urban|     Public| Experimental|
|     X2O| QOQTS|           25|Urban|     Public| Experimental|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
|     Q0E| ZOWMK|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



### df method

In [81]:
# 시골지역의 사립학교중 표준교육을 진행하는 학교들의 평균 학생수
# cdf.select(cdf.class_std_cnt) \
#     .where((cdf.loc=="Rural") &
#            (cdf.school_type=="Non-public") &
#            (cdf.teaching_type=="Standard")) \
#     .show()

cdf.where((cdf.loc=="Rural") &
           (cdf.school_type=="Non-public") &
           (cdf.teaching_type=="Standard")) \
    .show()

cdf.where((cdf.loc=="Rural") &
           (cdf.school_type=="Non-public") &
           (cdf.teaching_type=="Standard")) \
    .select(avg("class_std_cnt")) \
    .show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     MDS|  NULL|           18|Rural| Non-public|     Standard|
|     J6X| VHDHF|           16|Rural| Non-public|     Standard|
+--------+------+-------------+-----+-----------+-------------+

+------------------+
|avg(class_std_cnt)|
+------------------+
|              17.0|
+------------------+



In [96]:
# 시골지역의 사립학교중 표준교육을 진행하는 학교의 학급들의 평균 학생수보다 학생수가 더 많은 도시 지역의 공립학교면서 특수교육을 진행하는 학교의 모든 정보를 추출하시오
cdf.select("*") \
    .where((cdf.loc=="Urban") &
           (cdf.school_type=="Public") &
           (cdf.teaching_type=="Experimental") &
           (cdf.class_std_cnt > (
               cdf.where((cdf.loc=="Rural") &
                        (cdf.school_type=="Non-public") &
                        (cdf.teaching_type=="Standard")) \
                    .select(avg("class_std_cnt")) \
                    .collect()[0][0]
           )
           )
          ) \
    .show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     X6Z| CUQAM|           24|Urban|     Public| Experimental|
|     W8A| GOOBU|           26|Urban|     Public| Experimental|
|     0N7| QOQTS|           28|Urban|     Public| Experimental|
|     X2O| QOQTS|           25|Urban|     Public| Experimental|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
|     Q0E| ZOWMK|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+

